<a id='top'></a>
# calwebb_coron3
---
In this notebook, we run `calwebb_coron3` using a custom association file.

We require Stage 2b ("calints") files for:
 - background-subtracted PSF reference star exposures
 - background-subtracted science target exposures

**Author**: Jonathan Aguilar (jaguilar@stsci.edu) | **Latest Update**: Nov 2 Oct 2023

* [Introduction](#intro)
* [Pipeline Resources and Documentation](#resources)
* [Imports](#imports)
* [Convenience tools](#convenience_tools)
* [File selection](#file_selection)
* [Run Stage 3](#coron3)

<a id='intro'></a>
## Introduction

The Stage 3 JWST pipeline for coronagraphy takes photometrically calibrated `calints` pipeline products as input, and performs the following steps:

- outlier detection
- reference stacking and alignment
- PSF subtraction with the KLIP algorithnm
- Image resampling and World Coordinate System registration

Each input `calints` dataproduct consists of a 3-D cube that has units of MJy/Sr. This notebook breaks the calwebb_coron3 (also called Coron3Pipeline) pipeline class into steps, runs each step independently, and examines the output. It demonstrates how to change the parameters used to execute each step.



<a id='resources'></a>
## Pipeline resources and documentation

Documentation on `calwebb_coron3` and the steps run on MIRI coronagraphy data specifically can be found here: https://jwst-pipeline.readthedocs.io/en/latest/jwst/pipeline/calwebb_coron3.html.

<a id='imports'></a>
## Imports

In [ ]:
import os
from collections import OrderedDict
from pathlib import Path

In [ ]:
import numpy as np

In [ ]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from astropy.io import fits

<a id='convenience_tools'></a>
## Convenience tools

Environment paths and functions that make life easier.

First, set up a local CRDS directory. When the pipeline pulls a reference file from CRDS for the first time, it will write a copy to this directory. All subsequent reads of the reference file will redirect to the local directory instead of sending the file again over the network.

Use the local repository of reference files because it's faster

See https://jwst-pipeline.readthedocs.io/en/latest/jwst/user_documentation/reference_files_crds.html#crds

In [ ]:
os.environ['CRDS_PATH'] = '/Volumes/agdisk/crds/'
# os.environ['CRDS_PATH'] = ''
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

## Pipeline imports

In [ ]:
import jwst
jwst.__version__

In [ ]:
from jwst import datamodels
from jwst.datamodels import dqflags

In [ ]:
from jwst.pipeline import Coron3Pipeline

Advanced users - uncomment the cell below and specify the context if you have a specific combination of reference files you want to use

In [ ]:
# os.environ['CRDS_CONTEXT'] = 'jwst_1140.pmap'

In [ ]:
# some plot formatting
mpl.rcParams['image.origin'] = "lower"

In [ ]:
# you may need to run this command twice for plots to pop up correctly
# %matplotlib auto
%matplotlib inline

<a id="file_selection"></a>
## Collect Stage 2 products

The cells below will collect the Stage 2 files generated by the `calwebb_image2-all_exposures.ipynb` notebook. If you have run that notebook already, you can continue on with the current notebook. If you have not run that notebook but would like to use the same example data, you can retrieve the example exposures directly from MAST using the code snippet below. 

If you have your own exposures you would like to use, please prodide them as a list to the `filenames` variable. 

<div class="alert alert-block alert-info">
Snippet for downloading Stage 2 ERS-1386 data:

```
filenames = [
    'jw01386008001_04101_00001_mirimage_calints.fits',  # Science target, roll 1
    'jw01386009001_04101_00001_mirimage_calints.fits',  # Science target, roll 2
    'jw01386007001_04101_00001_mirimage_calints.fits',  # PSF reference star observations
    'jw01386007001_04101_00002_mirimage_calints.fits',
    'jw01386007001_04101_00003_mirimage_calints.fits',
    'jw01386007001_04101_00004_mirimage_calints.fits',
    'jw01386007001_04101_00005_mirimage_calints.fits',
    'jw01386007001_04101_00006_mirimage_calints.fits',
    'jw01386007001_04101_00007_mirimage_calints.fits',
    'jw01386007001_04101_00008_mirimage_calints.fits',
    'jw01386007001_04101_00009_mirimage_calints.fits',
]
from astroquery.mast import Observations
for filename in filenames:
    Observations.download_file(f"mast:JWST/product/{filename}", local_path= f"./stage3/input/{filename}")
example_cal_file = f"./stage3/input/{filenames[0]}"
```

</div>


In [ ]:
filenames = sorted(Path("stage2/output-asn_all/").glob("*calints.fits"))

### Get the rateints files



Let's do a quick inspection of one of the files.

In [ ]:
example_cal_file = filenames[0]

In [ ]:
# print some basic information
fits.info(example_cal_file)

In [ ]:
# a simple plot
fig, ax = plt.subplots(1, 1)
img = np.nanmean(fits.getdata(example_cal_file, 1), axis=0)
imlims = dict(zip(['vmin', 'vmax'], np.nanquantile(img, [0.01, 0.99])))
ax.imshow(img, **imlims, origin='lower')

In [ ]:
# Sort the files into science or PSF target, using the IS_PSF header keyword
files = {
    'sci': [],
    'psf': []
}

for f in filenames:
    is_psf = fits.getval(f, 'IS_PSF', 0)
    if is_psf == True:
        files['psf'].append(str(f.resolve()))
    else: # is_psf == False
        files['sci'].append(str(f.resolve()))
        
files

<a id='image2'></a>
## Run calwebb_coron3

To run calwebb_coron3, with have to generate an association file

The steps, in order of execution, are:


We're going to write out the results of each step to disk, and also keep a copy in memory in the `results` dict generated in the cell below.

In [ ]:
from jwst import associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list

In [ ]:
for f in files['sci'] + files['psf']:
    keys = ['SUBARRAY', 'IS_PSF', 'OBSERVTN', 'TARGNAME']
    vals = '\t'.join(str(fits.getval(f, k, 0)) for k in keys)
    print(vals)

In [ ]:
# to generate a proper background-subtracting association file, 
# first make one with just the science files and then add the background files
# by hand
asn = asn_from_list.asn_from_list(files['sci'], 
                                  rule=DMS_Level3_Base, 
                                  product_name='test')

for f in files['psf']:
    asn['products'][0]['members'].append({'expname': str(f), 'exptype': 'psf'})

In [ ]:
asn

#### 

In [ ]:
# write this out to a json file
with open('stage3/coron3_asn.json', 'w') as fp:
    fp.write(asn.dump()[1])

## Run Coron3


`calwebb_coron3` executes 5 different steps:
- `outlier_detection`
    Check for cosmic rays in PSF and science targets, and set the DQ flag appropriately. This has also been done during the Stage 1 `jump` step, but comparing multiple associated exposures can often be more senstitive. 
    - step-specific arguments (see https://jwst-pipeline.readthedocs.io/en/latest/jwst/outlier_detection/arguments.html for documentation):
        - `weight_type ['ivm']`
        - `pixfrac [1.0]`
        - `kernel ['square']`
        - `fillval ['INDEF']`
        - `nlow [0]`:
        - `nhigh [0]`
        - `maskpt [0.7]`:
        - `grow [1]`
        - `snr ['5.0 4.0']`
        - `scale ['1.2 0.7']`
        - `backg [0.0]`
        - `kernel_size ['7 7']`
        - `threshold_percent [99.8]`
        - `ifu_second_check [False]`
        - `save_intermediate_results [False]`
        - `resample_data [True]`
        - `good_bits ['~DO_NOT_USE']`
        - `scale_detection [False]`
- `stack_refs`
    - This step takes the various reference exposures and stacks them into a single 3-D data cube, preparing them for the subsequent steps of the stage. No other changes are made.
    - step-specific arguments: None
- `align_refs`
    - Computes offsets between science target images and reference PSF images, and shifts the PSF images into alignment.
    - step-specific arguments:
        - `median_box_length [3]`: box size used to median-replace bad pixels
        - `bad_bits ['DO_NOT_USE']`: DQ values to consider when median-replacing bad pixels
- `klip`
    - Generate a model PSF from the references using the KLIP algorithm and subtract it from the science data
    - step-specific arguments:
        - `truncate [50]`: The maximum number of KL modes to use
- `resample`
    - This routine will resample each input 2D image based on the WCS and distortion information, and will combine multiple resampled images into a single undistorted product.
    - step-specific arguments (see https://jwst-pipeline.readthedocs.io/en/latest/jwst/resample/arguments.html for documentation): 
        - `pixfrac [1.0]`
        - `kernel ['square']`
        - `fillval ['INDEF']`
        - `weight_type ['ivm']`
        - `output_shape [None]`
        - `crpix [None]`
        - `crval [None]`
        - `rotation [None]`
        - `pixel_scale_ratio [1.0]`
        - `pixel_scale [None]`
        - `output_wcs ['']`
        - `single [False]`
        - `blendheaders [True]`



The parameter dictionary - available with the command `param_dict = Coron3Pipeline().get_pars()` - contains the parameters and default values for each step. You can see the parameters for a particular step with `param_dict['steps'][{step_name}]`. In this example we are going to change the `truncate` parameter of the `klip` step from 50 to 25.

In [ ]:
# you can get a list of available options with `Coron3Pipeline().get_pars()`
params = {
    'output_dir': "./stage3/output/",
    'save_results': True
    'steps': {
        'klip': {'truncate': 25},
    }
}

cor3 = Coron3Pipeline().call(
    "stage3/coron3_asn.json",
    **params,
)

## Examine output

In [ ]:
%matplotlib inline

In [ ]:
imgs = {'roll1': datamodels.open("./stage3/output/jw01386008001_04101_00001_mirimage_a3001_psfsub.fits").data.copy(),
        'roll2': datamodels.open("./stage3/output/jw01386009001_04101_00001_mirimage_a3001_psfsub.fits").data.copy(),
        'combo': datamodels.open("./stage3/output/test_i2d.fits").data.copy()}

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 8))
vmin, vmax = np.nanquantile(np.concatenate(list([i.ravel() for i in imgs.values()])), [0.05, 0.95])
for i, roll in enumerate(imgs.keys()):
    img = imgs[roll]
    while img.ndim > 2:
        img = np.nanmean(img, axis=0)
    ax = axes[i]
    ax.set_title(roll)
    ax.imshow(img, vmin=vmin, vmax=vmax)

## Overlay sky coordinates

Overlay the RA and Dec grid over the combined rolls

In [ ]:
from astropy.wcs import WCS

In [ ]:
with fits.open("stage3/output/test_i2d.fits") as f:
    wcs = WCS(f[1].header)

In [ ]:
from astropy import units
from astropy.coordinates import SkyCoord, Distance
from astropy import time

In [ ]:
# The star coordinates at the time of observation are in the header
exp_file = files['sci'][0]
targ_ra = fits.getval(exp_file, 'TARG_RA', 0)
targ_dec = fits.getval(exp_file, 'TARG_DEC', 0)
starcoord = SkyCoord(targ_ra, targ_dec, unit='deg', frame='icrs')

In [ ]:
fig, ax = plt.subplots(1, 1, subplot_kw={'projection':wcs})
vmin, vmax = np.nanquantile(imgs['combo'], [0.01, 0.99])
ax.imshow(imgs['combo'], vmin=vmin, vmax=vmax)
ax.scatter(*wcs.world_to_pixel(starcoord),
           marker='x', s=100, c='w')
ax.grid(True)